# Fouille données

In [ ]:
!pip install pandas numpy matplotlib seaborn folium scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from collections import Counter

%matplotlib inline
sns.set(style="whitegrid")

In [ ]:
# Chargement du fichier
# on_bad_lines='skip' permet de sauter les lignes mal formées sans planter
# On enlève engine='python' car le moteur par défaut (C) est souvent plus tolérant

df = pd.read_csv('flickr_data2.csv', on_bad_lines='skip')

print("Données brutes :")
display(df.head())

In [ ]:
print("Colonnes du DataFrame :")
print(df.columns.tolist())
print("\nInfo du DataFrame :")
df.info()

In [ ]:
# Histogramme de la colonne année (date_taken_year)
plt.figure(figsize=(10, 6))
sns.histplot(df[' date_taken_year'], bins=20, kde=True)
plt.title('Histogramme des années de prise des photos')
plt.xlabel('Année')
plt.ylabel('Nombre de photos')
plt.show()

In [ ]:
# Vérifier la plage des années
print("Année minimale :", df[' date_taken_year'].min())
print("Année maximale :", df[' date_taken_year'].max())
print("Années uniques :", sorted(df[' date_taken_year'].unique()))

In [ ]:
# Filtrer les données pour les années entre 2000 et 2026
df_filtered = df[(df[' date_taken_year'] >= 2000) & (df[' date_taken_year'] <= 2026)]

# Histogramme précis des années filtrées
plt.figure(figsize=(12, 6))
sns.histplot(df_filtered[' date_taken_year'], bins=range(2000, 2027), binwidth=1)
plt.title('Histogramme des années de prise des photos (2000-2026)')
plt.xlabel('Année')
plt.ylabel('Nombre de photos')
plt.xticks(range(2000, 2027, 2))  # Étiquettes tous les 2 ans pour lisibilité
plt.show()

In [ ]:
# Statistiques descriptives des colonnes numériques
print("Statistiques descriptives :")
display(df.describe())

# Nombre de valeurs manquantes par colonne
print("\nValeurs manquantes par colonne :")
display(df.isnull().sum())